In [1]:
from __future__ import division, print_function
import sys, os, glob, time, warnings, gc
# import matplotlib
# matplotlib.use('Agg')
import matplotlib.pyplot as plt
import numpy as np
from astropy.table import Table, vstack, hstack
import fitsio
from astropy.io import fits
import healpy as hp
from astropy import wcs

from scipy.ndimage.filters import gaussian_filter
from pathlib import Path
from multiprocessing import Pool

from scipy import stats

In [2]:
params = {'legend.fontsize': 'x-large',
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large',
         'figure.facecolor':'w'} 
plt.rcParams.update(params)

In [3]:
image_dir = '/global/cfs/cdirs/cosmo/staging/'
surveyccd_path = '/global/project/projectdirs/cosmo/work/legacysurvey/dr9m/survey-ccds-90prime-dr9.fits.gz'

# image_path_list = glob.glob(os.path.join(image_dir, '*ooi*.fits.fz'))
ccd = Table(fitsio.read(surveyccd_path))

In [4]:
mask = ccd['ccd_cuts']==0
ccd = ccd[mask]

In [5]:
# Only keep unique exposures
_, idx = np.unique(ccd['expnum'], return_index=True)
ccd = ccd[idx]

In [13]:
ccdnum_list = [1, 2, 3, 4]
ccd_ra = [0.2813, 0.2813, -0.2813, -0.2813]
ccd_dec = [0.263, -0.263, 0.263, -0.263]

binsize = 4
pix_size = 0.454/3600*binsize
vrange = 1.

In [22]:
mask = np.array(['CP20160111' in tmp for tmp in ccd['image_filename']])
print(np.sum(mask))
idx = np.where(mask)[0]

# idx = np.random.choice(idx, size=10, replace=None)
# idx.sort()

143


In [19]:
ccd[mask]

image_filename,image_hdu,camera,expnum,plver,procdate,plprocid,ccdname,object,propid,filter,exptime,mjd_obs,airmass,fwhm,width,height,ra_bore,dec_bore,crpix1,crpix2,crval1,crval2,cd1_1,cd1_2,cd2_1,cd2_2,yshift,ra,dec,skyrms,sig1,ccdzpt,zpt,ccdraoff,ccddecoff,ccdskycounts,ccdskysb,ccdrarms,ccddecrms,ccdphrms,phrms,ccdnastrom,ccdnphotom,ccd_cuts
bytes120,int16,bytes7,int64,bytes8,bytes19,bytes7,bytes4,bytes35,bytes10,bytes1,float32,float64,float32,float32,int16,int16,float64,float64,float32,float32,float64,float64,float32,float32,float32,float32,bool,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,int16,int64
90prime/CP/V2.3/CP20160111/ksb_160112_035141_ooi_g_ls9.fits.fz,1,90prime,73990039,V2.3,2019-08-30T21:55:11,95759fc,CCD1,51531,2015A-0801,g,173.4,57399.160901,1.55,4.17,4032,4096,94.5989166667,79.00325,4096.24,4283.25,94.5989166667,79.00325,3.59198e-07,-0.000125893,-0.000125909,-2.36425e-07,False,96.1079564098,79.2624860269,0.17818,0.00909909,25.7321,25.7379,-0.0362624,-0.0424103,5.56566,23.3563,0.0627695,0.0695573,0.0146034,0.014147,1891,2096,0
90prime/CP/V2.3/CP20160111/ksb_160112_035530_ooi_g_ls9.fits.fz,1,90prime,73990040,V2.3,2019-08-30T21:53:57,95759fc,CCD1,52091,2015A-0801,g,149.2,57399.1635531,1.59,4.21,4032,4096,99.6151666667,80.0025,4096.25,4283.1,99.6151666667,80.0025,4.32308e-07,-0.000125902,-0.000125913,-3.20826e-07,False,101.27642015,80.2615577843,0.19243,0.00992387,25.728,25.7304,-0.0274514,-0.020571,5.6555,23.3389,0.0556648,0.061826,0.0181553,0.0160955,1817,2014,0
90prime/CP/V2.3/CP20160111/ksb_160112_035949_ooi_g_ls9.fits.fz,1,90prime,73990041,V2.3,2019-08-30T21:54:04,95759fc,CCD1,42431,2015A-0801,g,184.5,57399.1665485,1.29,3.35,4032,4096,91.3121666667,66.0046944444,4096.31,4283.23,91.3121666667,66.0046944444,-1.02618e-07,-0.000125879,-0.000125897,2.48084e-07,False,92.0128928184,66.2648904821,0.170111,0.00846161,25.7765,25.7784,-0.0356295,-0.041754,5.39952,23.3892,0.0495499,0.0551162,0.0185613,0.0173304,2897,3239,0
90prime/CP/V2.3/CP20160111/ksb_160112_040402_ooi_g_ls9.fits.fz,1,90prime,73990042,V2.3,2019-08-30T21:55:34,95759fc,CCD1,64531,2015A-0801,g,235.8,57399.1694698,1.2,3.09,4032,4096,90.442,58.0052777778,4096.39,4283.31,90.442,58.0052777778,-2.01457e-07,-0.000125876,-0.000125888,3.55459e-07,False,90.9786203084,58.2656780777,0.154601,0.00743149,25.8074,25.8065,-0.0349491,-0.0379221,5.60408,23.3488,0.0445589,0.0484542,0.0148107,0.0169106,3343,3759,0
90prime/CP/V2.3/CP20160111/ksb_160112_040959_ooi_g_ls9.fits.fz,1,90prime,73990043,V2.3,2019-08-30T21:53:57,95759fc,CCD1,64211,2015A-0801,g,223.2,57399.1736005,1.18,3.57,4032,4096,91.5415416667,56.0050555556,4096.33,4283.23,91.5415416667,56.0050555556,-2.19544e-07,-0.000125874,-0.000125882,3.76738e-07,False,92.0498283148,56.2654718101,0.165989,0.00769045,25.8122,25.8108,-0.0424507,-0.0427103,5.71194,23.3281,0.0524288,0.054972,0.0164523,0.0127528,3303,3819,0
90prime/CP/V2.3/CP20160111/ksb_160112_041437_ooi_g_ls9.fits.fz,1,90prime,73990044,V2.3,2019-08-30T21:54:05,95759fc,CCD1,64371,2015A-0801,g,236.8,57399.1768284,1.18,3.41,4032,4096,90.76125,57.0041388889,4096.46,4283.34,90.76125,57.0041388889,-2.12644e-07,-0.000125875,-0.000125884,3.68375e-07,False,91.2832484186,57.2645558571,0.153763,0.00740939,25.8095,25.8108,-0.0520392,-0.044229,5.65576,23.3388,0.0605847,0.0557654,0.0176869,0.0151013,3410,3897,0
90prime/CP/V2.3/CP20160111/ksb_160112_041930_ooi_g_ls9.fits.fz,1,90prime,73990045,V2.3,2019-08-30T21:55:08,95759fc,CCD1,63891,2015A-0801,g,250.0,57399.1802132,1.16,3.23,4032,4096,93.2848333333,54.0044444444,4096.3,4283.37,93.2848333333,54.0044444444,-2.35492e-07,-0.000125873,-0.000125882,3.93561e-07,False,93.7682303956,54.2648931708,0.153948,0.00726022,25.8137,25.8149,-0.0544589,-0.0489172,5.80132,23.3112,0.0632254,0.0585125,0.0145295,0.0135563,3527,3949,0
90prime/CP/V2.3/CP20160111/ksb_160112_042436_ooi_g_ls9.fits.fz,1,90prime,73990046,V2.3,2019-08-30T21:56:37,95759fc,CCD1,63731,2015A-0801,g,250

In [27]:
from matplotlib.backends.backend_pdf import PdfPages
with PdfPages('plots2/test_20160111.pdf') as pdf:

    for ccd_index in idx[::5]:

        fn = ccd['image_filename'][ccd_index].strip()
        expnum = ccd['expnum'][ccd_index]
        band = ccd['filter'][ccd_index]
        print(ccd_index, band, expnum)

        plt.figure(figsize=(14, 12))

        for ii, ccdnum in enumerate(ccdnum_list):

            try:
                img = fits.getdata('/global/cfs/cdirs/cosmo/staging/'+fn, extname='CCD'+str(ccdnum))
            except:
                print('Failure loading {}'.format('/global/cfs/cdirs/cosmo/staging/'+fn))
                continue

            # naive sky estimation
            mask = (img<np.percentile(img.flatten(), 95))
            median_sky = np.median(img[mask].flatten())
            img = img - median_sky

            ################ downsize image ################

            trim_size_x = img.shape[1] % binsize
            trim_size_y = img.shape[0] % binsize
            img = img[:(img.shape[0]-trim_size_y), :(img.shape[1]-trim_size_x)]
            # to ignore NAN values, use np.nanmean
            img = np.nanmean(np.nanmean(img.reshape((img.shape[0]//binsize, binsize, img.shape[1]//binsize,-1)), axis=3), axis=1)

            ################################################

            ysize, xsize = img.shape        
            ra, dec = ccd_ra[ii], ccd_dec[ii]

            img[~np.isfinite(img)] = 0
            # img = gaussian_filter(img, 4, mode='reflect', truncate=3)
            fig = plt.imshow(img.T, cmap='seismic', vmin=-vrange, vmax=vrange, 
                       extent=(ra+ysize*pix_size/2, ra-ysize*pix_size/2, dec-xsize*pix_size/2, dec+xsize*pix_size/2))

        plt.axis([0.55, -0.55, -0.55, 0.55])
        plt.axis('off')
        fig.axes.get_xaxis().set_visible(False)
        fig.axes.get_yaxis().set_visible(False)
        plt.colorbar(fraction=0.04, pad=0.04)
        plt.title('expnum={}  ccd_cuts={}  {}'.format(expnum, ccd['ccd_cuts'][ccd_index], fn))
        plt.tight_layout()
        # plt.savefig(os.path.join(plot_dir, os.path.basename(image_path).replace('.fits.fz', '.png')))
        pdf.savefig()
        plt.close()

        ######################## Original mask ########################
        
        # mask_path = '/global/u2/r/rongpu/temp/90prime_junk/mask_{}.npz'.format(expnum)
        # img_mask = np.load(mask_path)
        
        fn_ood = fn.replace('_ooi_', '_ood_')
        mask_path = os.path.join(image_dir, fn_ood)
        
        plt.figure(figsize=(14, 12))

        for ii, ccdnum in enumerate(ccdnum_list):

            try:
                img = fits.getdata('/global/cfs/cdirs/cosmo/staging/'+fn, extname='CCD'+str(ccdnum))
            except:
                print('Failure loading {}'.format('/global/cfs/cdirs/cosmo/staging/'+fn))
                continue

            # naive sky estimation
            mask = (img<np.percentile(img.flatten(), 95))
            median_sky = np.median(img[mask].flatten())
            img = img - median_sky
            
            # if 'CCD'+str(ccdnum) in img_mask.keys():
            #     img[img_mask['CCD'+str(ccdnum)]] = 0.
            ood = fits.getdata(mask_path, extname='ccd'+str(ccdnum))
            img[ood!=0] = 0.

            ################ downsize image ################

            trim_size_x = img.shape[1] % binsize
            trim_size_y = img.shape[0] % binsize
            img = img[:(img.shape[0]-trim_size_y), :(img.shape[1]-trim_size_x)]
            # to ignore NAN values, use np.nanmean
            img = np.nanmean(np.nanmean(img.reshape((img.shape[0]//binsize, binsize, img.shape[1]//binsize,-1)), axis=3), axis=1)

            ################################################

            ysize, xsize = img.shape        
            ra, dec = ccd_ra[ii], ccd_dec[ii]

            img[~np.isfinite(img)] = 0
            # img = gaussian_filter(img, 4, mode='reflect', truncate=3)
            fig = plt.imshow(img.T, cmap='seismic', vmin=-vrange, vmax=vrange, 
                       extent=(ra+ysize*pix_size/2, ra-ysize*pix_size/2, dec-xsize*pix_size/2, dec+xsize*pix_size/2))

        plt.axis([0.55, -0.55, -0.55, 0.55])
        plt.axis('off')
        fig.axes.get_xaxis().set_visible(False)
        fig.axes.get_yaxis().set_visible(False)
        plt.colorbar(fraction=0.04, pad=0.04)
        plt.title('expnum={}  ccd_cuts={}  {}'.format(expnum, ccd['ccd_cuts'][ccd_index], fn))
        plt.tight_layout()
        # plt.savefig(os.path.join(plot_dir, os.path.basename(image_path).replace('.fits.fz', '.png')))
        pdf.savefig()
        plt.close()

211 g 73990039
216 g 73990044
221 g 73990049
226 g 73990054
231 g 73990059
236 g 73990064
241 g 73990069
246 g 73990074
251 g 73990079
256 g 73990084
261 g 73990089
266 g 73990094
271 g 73990099
276 g 73990104
281 g 73990109
286 g 73990114
291 g 73990119
296 g 73990124
301 g 73990129
306 g 73990134
311 g 73990139
316 g 73990144
321 g 73990149
326 g 73990157
331 g 73990162
336 g 73990167
341 g 73990172
346 g 73990177
351 g 73990182


In [28]:
# idx = np.where(ccd['expnum']==79240048)[0][0]
idx1 = np.concatenate([np.arange(idx.min()-6, idx.min()), np.arange(idx.max()+1, idx.max()+6)])
print(idx1)

from matplotlib.backends.backend_pdf import PdfPages
with PdfPages('plots2/test_20160111_neighbors.pdf') as pdf:

    for ccd_index in idx1:

        # from matplotlib.backends.backend_pdf import PdfPages
        # with PdfPages('plots/90prime_images-ccd4.pdf') as pdf:
        # for ccd_index in range(idx-20, idx+20):

        expnum = ccd['expnum'][ccd_index]
        band = ccd['filter'][ccd_index]
        print(ccd_index, band, expnum)
        fn = ccd['image_filename'][ccd_index]

        plt.figure(figsize=(14, 12))

        for ii, ccdnum in enumerate(ccdnum_list):

            try:
                img = fits.getdata('/global/cfs/cdirs/cosmo/staging/'+fn, extname='CCD'+str(ccdnum))
            except:
                print('Failure loading {}'.format('/global/cfs/cdirs/cosmo/staging/'+fn))
                continue

            # naive sky estimation
            mask = (img<np.percentile(img.flatten(), 95))
            median_sky = np.median(img[mask].flatten())
            img = img - median_sky

            ################ downsize image ################

            trim_size_x = img.shape[1] % binsize
            trim_size_y = img.shape[0] % binsize
            img = img[:(img.shape[0]-trim_size_y), :(img.shape[1]-trim_size_x)]
            # to ignore NAN values, use np.nanmean
            img = np.nanmean(np.nanmean(img.reshape((img.shape[0]//binsize, binsize, img.shape[1]//binsize,-1)), axis=3), axis=1)

            ################################################

            ysize, xsize = img.shape        
            ra, dec = ccd_ra[ii], ccd_dec[ii]

            img[~np.isfinite(img)] = 0
            # img = gaussian_filter(img, 4, mode='reflect', truncate=3)
            fig = plt.imshow(img.T, cmap='seismic', vmin=-vrange, vmax=vrange, 
                       extent=(ra+ysize*pix_size/2, ra-ysize*pix_size/2, dec-xsize*pix_size/2, dec+xsize*pix_size/2))

        plt.axis([0.55, -0.55, -0.55, 0.55])
        plt.axis('off')
        fig.axes.get_xaxis().set_visible(False)
        fig.axes.get_yaxis().set_visible(False)
        plt.colorbar(fraction=0.04, pad=0.04)
        plt.title('expnum={}  ccd_cuts={}  {}'.format(expnum, ccd['ccd_cuts'][ccd_index], fn))
        plt.tight_layout()
        # plt.savefig(os.path.join(plot_dir, os.path.basename(image_path).replace('.fits.fz', '.png')))
        pdf.savefig()
        plt.close()

[205 206 207 208 209 210 354 355 356 357 358]
205 g 73910073
206 g 73910074
207 g 73910075
208 g 73910076
209 g 73910077
210 g 73910078
354 g 74000109
355 g 74000110
356 g 74000111
357 g 74000112
358 g 74000113
